# Navier Stokes with energy input from the bottom edge

This code runs a Navier Stokes simulator with particle movement included. The evolution of the fluid will be created by a force in the y direction
along the top edge (y=1). The magnitude of the force will be Gaussian with a peak at x=0.5, and grow over time. Guassian noise will be added to this force. 


                             _________________________
                         y=1|                         |           |
                            |                         |           |     Force downwards at each point due to gravity
                            |    O                    |           v
                            |               O         |
                            |                         |
                            |        O                |
                            |                         |
                            |               O         |
                         y=0|_________________________|
                         x=0                           x=1
                             ^  ^  ^  ^  ^  ^  ^  ^  ^  
                             :  |  |  |  |  |  |  |  :      Velocity on bottom edge due to heating
                                   :  |  |  |  :
                                         :    
Boundary Conditions:
* The boundary conditions for velocity are Dirichlet, meaning that the value of the velocities on the boundary are known and enforced. For the left, 
   right and top edges, the velocities are set to 0, in line with the "no slip" condition of viscous fluids. Along the bottom edge, the velocity of the fluid is set to the to value generated by the force explained above, with some noise added.
* The boundary condition for the pressure is Neumann, meaning that the gradient of the pressure across the boundary is known and enforced. This is set as 0.

Initial Conditions:
* The initial conditions for velocity are a generated Divergence Free field, generated using the code provided by Niall Jeffrey's in the python notebook "divergence_free_field.py". 
* The pressure at each point is generated from the standard normal distribution.
* The x and y coordinates of the particles are generated from a uniform distribution between 0 and 1. 

In [ ]:
from navier_stokes_2D import progress_timestep_with_particles, kinetic_energy 
from divergence_free_field import k2g_fft, compute_spectrum_map, gaussian_mock
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import math
import time as time
import jax.config as config
import numpy as np

import imageio

config.update("jax_debug_nans", True)
config.update("jax_enable_x64", True)

In [ ]:
"""Determine Constants."""

TIMESTEPS = 200000
NUM_POINTS = 128   # How many points to split x and y axis into.
dt = 0.0001        # The size of the timesteps.

IMAGE_SCALAR = math.ceil(NUM_POINTS / 32)       # This is used to ensure the output images aren't too dense, so they can actually be made out. 

element_length = 1 / (NUM_POINTS - 1)           # The distance between points on the x and y axis.
    
"""Define the Initial Conditions and Boundary Conditions."""

# Make a meshgrid to create our domain.
x = jnp.linspace(0, 1, NUM_POINTS)
y = jnp.linspace(0, 1, NUM_POINTS)

X, Y = jnp.meshgrid(x, y)

# This makes the velocity along the bottom shaped like a Gaussian distribution. It is not added to u_bound yet as it will increase as time goes on,
# similar to a heat source heating up the fluid and increasing its velocity. Modify this to change the energy input to the system.
gaussian_curve = jnp.linspace(-5, 5, NUM_POINTS)
gaussian_curve =  2 * jnp.exp(- jnp.square(gaussian_curve))

# Define the boundary conditions.
u_bound = jnp.zeros_like(X)
v_bound = jnp.zeros_like(X)

# A matrix of just 1 along the bottom row must also be created so that at each timestep it can be multiplied by a matrix of normally distributed values
# to simulate the noise.
bottom_ones = jnp.zeros_like(X)
bottom_ones = bottom_ones.at[0, :].set(1)

# Define the initial conditions. The next 4 lines define the divergence free field that will be generated.
karray = np.arange(NUM_POINTS)
pk = np.exp(-karray * karray / 5)
spectrum_map = compute_spectrum_map(pk, NUM_POINTS)
field = gaussian_mock(spectrum_map.flatten(), NUM_POINTS).T

# Generate the divergence free field. The velocities generated tend to be very small, so multiply the velocities by 10000
# so that the velocity of the fluid is great enough to move the particles by a noticable amount given the size of the timesteps. 
u_prev, v_prev = k2g_fft(field*0, field, dx=1, pad=False)
u_prev = 20000 * u_prev 
v_prev = 20000 * v_prev 

# Pressure can be picked from a normal distribution at each point.
p_prev = np.random.normal(u_bound)

# Generated the initial x and y coordinates of the particle. These can be anywhere in the fluid, so generate these uniformly. 
# This will generate 10 particles.
x_prev = np.random.uniform(0, 1, 10)
y_prev = np.random.uniform(0, 1, 10)

# The initial particle velocities can be set to 0.
dx_dt_prev = jnp.zeros_like(x_prev)
dy_dt_prev = jnp.zeros_like(x_prev)

# The acting force on the fluid will be gravity. This will just be a uniform vector field pointing downwards. 
f_x = jnp.zeros_like(u_prev)
# Change 2000 to a different number to change the strength of gravity. 
f_y = - 10 * jnp.ones_like(u_prev)

#This produces the plot of the initial conditions.
plt.figure()
plt.contourf(X[::IMAGE_SCALAR, ::IMAGE_SCALAR], Y[::IMAGE_SCALAR, ::IMAGE_SCALAR], p_prev[::IMAGE_SCALAR, ::IMAGE_SCALAR], 100, cmap="coolwarm")
plt.colorbar()
plt.quiver(X[::IMAGE_SCALAR, ::IMAGE_SCALAR], Y[::IMAGE_SCALAR, ::IMAGE_SCALAR], u_prev[::IMAGE_SCALAR, ::IMAGE_SCALAR], v_prev[::IMAGE_SCALAR, ::IMAGE_SCALAR], color="black")
plt.show()

In [ ]:
"""Run training loop and save images."""
    
for i in range(TIMESTEPS):
    # update the velocities and pressure.
    u_prev, v_prev, p_prev, x_prev, y_prev, dx_dt_prev, dy_dt_prev = progress_timestep_with_particles(u_prev, v_prev, p_prev, x_prev, 
                                                                                                      y_prev, dx_dt_prev, dy_dt_prev, 
                                                                                                      u_bound, v_bound, element_length,
                                                                                                      f_x=f_x, f_y=f_y, drag_constant=100, 
                                                                                                      dt=dt, density=1., viscosity=0.01, 
                                                                                                      jacobi_iterations=50, bounce=True)
    
    # If it is currently less that 100000 timesteps, increase the y velocity along the top edge. Once 100000 timesteps have been reached, 
    # the velocity should stay around the same value but vary due to the noise. 
    if i >= 2000:
        # Generate a key for the noise.
        key = jax.random.PRNGKey(int(time.time()))
        # Increase the velocity. The second added term is to add noise to the field. 
        v_bound = 0.5 * v_bound.at[0, :].set(gaussian_curve) + 0.25 * bottom_ones * jax.random.normal(key=key, shape=jnp.shape(v_bound))
    
    
    # Print the time and kinetic energy every 1000 timesteps, and save an image of the current state.
    if i % 100 == 0:
        print(f"timestep: {i}, time: {i*dt}")
        print(f"Kinetic Energy = {kinetic_energy(u_prev, v_prev)}")
        plt.figure()
        plt.contourf(X, Y, p_prev, 100, cmap="coolwarm")
        plt.colorbar()
        plt.figtext(0.15, 0.9, str(i), ha="right")
        
        Q = plt.quiver(X[::IMAGE_SCALAR, ::IMAGE_SCALAR], Y[::IMAGE_SCALAR, ::IMAGE_SCALAR], u_prev[::IMAGE_SCALAR, ::IMAGE_SCALAR], v_prev[::IMAGE_SCALAR, ::IMAGE_SCALAR])
        plt.scatter(x_prev, y_prev, color="g")
        plt.margins(x=0, y=0)
        

        plt.savefig(f'img_{int(i/100)}.png',
                  transparent = False,  
                  facecolor = 'white'
               )
        plt.close()

In [ ]:
"""Make images into a gif."""
frames = []
for i in range(400):
  image = imageio.v2.imread(f'img_{int(i)}.png')
  frames.append(image)

imageio.mimsave('./example.gif', # output gif
                frames,          # array of input frames
                duration=0.4)         # optional: frames per second """